# Logistische Regression: Digit Classification

In diesem Notebook wollen wir anhand des Digits-Datensatzes Vorhersagen mit Hilfe einer logistischen Regression machen. In diesem Datensatz ist jeder Datenpunkt ein kleines Bild mit 8x8 Pixeln und stellt eine Ziffer zwischen 0 und 9 dar. Das Vorhersageproblem besteht darin, anhand der 64 Pixelwerte vorherzusagen um welche Zahl es sich handelt.
Eine nähere Beschreibung der Daten findet sich [hier](https://scikit-learn.org/stable/datasets/toy_dataset.html#digits-dataset).

### Laden des Digits-Datensatzes

Zuerst Laden wir den `Digits`-Datensatz:

In [ ]:
import pandas as pd
data = pd.read_csv("data/digits.csv")
print(data.shape)
data.head()

Ein Datenpunkt hat 64 Features. Jedes Feature steht für ein Pixel in einem 8x8 Bild und hat einen Wert zwischen `0` und `16`, welcher der Graustufe des Pixels entspricht. Für den Wert `0` ist das Pixel komplett schwarz, während ein Wert von `16` einem weißen Pixel entspricht. Das Label eines Datenpunkts ist die abgebildete Ziffer.

Die ersten 8 Features entsprechen den Pixeln der ersten Reihe des Bildes, die zweiten 8 Features der zweiten Reihe des Bildes usw.
Um das besser zu veranschaulichen können wir die Daten des ersten Datenpunktes in eine 8x8-Form bringen:

In [ ]:
import numpy as np
data_no_label = data.drop(["label"], axis=1)
line_1 = data_no_label.iloc[0].values
np.reshape(line_1, (8,8))

Um uns testweise ein paar Bilder anzuschauen können wir ein paar zufällig ausgewählte Datenpunkte mit Hilfe der `matplotlib`-Library plotten:

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import random

no_images = 15

plt.figure(figsize=(no_images, 1))
for index in range(0, no_images):
    
    # wähle zufälligen Datenpunkt aus
    image_index = random.randint(0, data.shape[0])
    features = data_no_label.iloc[image_index].values
    features_8x8 = np.reshape(features, (8,8))
    label = data.iloc[image_index]["label"]
    
    # plotte das 8x8 Bild
    plt.subplot(1, no_images, index + 1)
    plt.imshow(features_8x8, cmap=plt.cm.gray)
    plt.title(label, fontsize = 20)

### Binäre Klassifikation

Ziel ist es jetzt mit Hilfe der Features (Pixelwerte) das Label (die dargestellte Ziffer) vorherzusagen. Dazu wollen wir uns im ersten Schritt auf eine einfache binäre Klassifizierung beschränken und Vorhersagen ob auf dem Bild die Ziffer `0` zu sehen ist.

Im ersten Schritt unterteilen wir die Daten in Trainings- und Testdaten:

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_no_label, data["label"], test_size=0.2, random_state=0)
print("Trainingsdatenpunkte:", len(X_train))
print("Testdatenpunkte:", len(X_test))

Für die Klassifierzung hinsichtlich der Ziffer `0` müssen wir das Label jedes Datenpunktes umwandeln, so dass gilt:
- `Label "1"`: Die Ziffer ist eine `0`.
- `Label "0"`: Die Ziffer ist keine `0` (d.h. alle andere Ziffern).

Diese machen wir über folgende Funktion:

In [ ]:
def label_to_binary(digit, labels):
    binary_labels = []
    for label in labels:
        if(label == digit):
            binary_labels.append(1)
        else:
            binary_labels.append(0)
    return np.array(binary_labels)
    
y_train_0 = label_to_binary(0, y_train)
y_test_0 = label_to_binary(0, y_test)

print("Alte Labels:", y_train[:20].values)
print("Neue Labels:", y_train_0[:20])

Jetzt können wir das logistische Regressionsmodell trainieren.

In [ ]:
from sklearn.linear_model import LogisticRegression
logisticRegr = LogisticRegression(max_iter=1000) # Gradient Descent mit max. 1000 Schritten
logisticRegr.fit(X_train, y_train_0);

### Testen des Regressionsmodells

Schauen wir uns nun zwei Beispiele aus den Testdaten und die dazugehörige Vorhersage an:

In [ ]:
data_index = 0
data_point = X_test.iloc[[data_index]]
prediction = logisticRegr.predict(data_point)
print("Vorhersage auf Datenpunkt", data_index, "ist:", prediction)

Das Modell sagt die Klasse `0` vorraus (d.h. dass nicht die Ziffer 0 abgebildet ist). Schauen wir uns das richtige Label und den zugehörigen Plot an.

In [ ]:
plt.plot()
plt.imshow(np.reshape(data_point.values, (8,8)), cmap=plt.cm.gray)
plt.title(f'Ziffer: {y_test.iloc[data_index]}', fontsize = 20)

Die Vorhersage ist richtig (es ist nicht die Ziffer 0 auf dem Bild dargestellt).
Schauen wir uns ein zweites Beispiel an.

In [ ]:
data_index = 17
data_point = X_test.iloc[[data_index]]
prediction = logisticRegr.predict(data_point)
print("Vorhersage auf Datenpunkt", data_index, "ist:", prediction)

In [ ]:
plt.plot()
plt.imshow(np.reshape(data_point.values, (8,8)), cmap=plt.cm.gray)
plt.title(f"Ziffer: {y_test.iloc[data_index]}", fontsize = 20)

In diesem Fall sagt das Modell die Klasse `1` (= es ist eine Null auf dem Bild zu sehen) voraus und es ist auch tatsächlich eine `0` auf dem Bild, d.h. das Modell liegt richtig. 

Schauen wir uns die Performance auf allen Testdaten an. Dazu mache wir zuerst eine Vorhersage mit dem Modell auf allen Testdaten:

In [ ]:
predictions = logisticRegr.predict(X_test)

Im nächsten Schritt vergleich wir diese Vorhersagen mit dem wahren Label und berechnen die Accuracy:

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test_0, predictions)

Die Accuracy liegt bei diesem einfachen Modell bei 99 Prozent. Schauen wir uns Precision und Recall an.

In [ ]:
from sklearn.metrics import precision_score
precision_score(y_test_0, predictions)

In [ ]:
from sklearn.metrics import recall_score
recall_score(y_test_0, predictions)

Das Modell erkennt alle Nullen im Testdatensatz und ist dabei 93% präzise.
Es scheint also, dass das Problem eine Null in diesem Datensatz zu erkennen nicht sehr schwierig ist.

Da wir jedoch keine 100% Precision erreicht haben, macht das Modell noch ein paar Fehler. Schauen wir uns exemplarisch einen Fehler an:

In [ ]:
data_index = 117
data_point = X_test.iloc[[data_index]]
prediction = logisticRegr.predict(data_point)
print("Vorhersage auf Datenpunkt", data_index, "ist:", prediction)

In [ ]:
plt.plot()
plt.imshow(np.reshape(data_point.values, (8,8)), cmap=plt.cm.gray)
plt.title("Ziffer: " + str(y_test.iloc[data_index]), fontsize = 20)

Dieses Zwei hält das Modell fälschlicherweiße für eine Null.

### Feature Importance

Beim gerade trainierten Modell können wir uns jetzt noch die Feature-Importance anschauen, d.h. die Gewichte welche das Modell für die einzelnen Pixel gelernt hat anschauen.

In [ ]:
weights = logisticRegr.coef_[0]

In [ ]:
weights_8x8 = np.round(np.reshape(weights, (8,8)), decimals=3)
weights_8x8

In [ ]:
plt.imshow(weights_8x8)

In diesem Plot stehen grüne Fläche für ein Gewicht in der Nähe von 0, d.h. diese Pixel sind relativ irrelevant für die Klassifikation des Modells.
Gelbe Flächen stehen für positive Gewichte, d.h. ein positiver Pixelwert an dieser Stelle lässt das Modell eher in Richtung 0 tendieren.
Dunkle Flächen stehen für negative Gewichte, d.h. ein positiver Pixelwert an dieser Stelle lässt das Modell eher in Richting "keine 0" tendieren. 

### Multiklassen-Klassifikation

Im nächsten Schritt erweitern wir das Vorhersagemodell dahingehend, dass wir alle Ziffern erkennen wollen. Dies geschieht mit einer multinominalen logistischen Regression.

Dazu trainieren wir individuelle Modell für jede Ziffer von 0 bis 9.

In [ ]:
models = []
for i in range(0,10):
    logisticRegr = LogisticRegression(max_iter=1000)
    y_train_binary = label_to_binary(i, y_train)
    logisticRegr.fit(X_train, y_train_binary)
    models.append(logisticRegr)

Diese Modell nutzen wir jetzt bei der Vorhersage, in dem wir mit jedem Modell eine Vorhersage auf ein gegebens Bild machen und uns für die Vorhersage mit der höchsten Wahrscheinlichkeit entscheiden. Dazu brauchen wir (anstelle der vorhergesagten Klasse) die Vorhersagewahrscheinlichkeit eines Modells. Generell können wir mit der Methode `predict` direkt die Vorhersage der Klasse bekommen, während die `predict_proba`-Methode uns die Wahrscheinlichkeiten liefert:

In [ ]:
data_point = X_test.iloc[[0]]
predicted_class = logisticRegr.predict(data_point)
print("Vorhergesagte Klasse:", predicted_class)

probability = logisticRegr.predict_proba(data_point)
print(probability)
print("Wahrscheinlichkeit für Klasse 1:", probability[0][1])

Mit den trainierten Einzelmodellen können wir nun für einen Datenpunkt vorhersagen mit allen 10 Modell machen:

In [ ]:
data_index = 34
predictions = []
for index, model in enumerate(models):
    data_point = X_test.iloc[[data_index]]
    prediction = model.predict_proba(data_point)[0][1]
    print("Vorhersage mit Modell für Ziffer", index, "ist", prediction)
    predictions.append(prediction)

Und die Klasse vorhersagen welche die höchste Wahrscheinlichkeit hat:

In [ ]:
prediction = predictions.index(max(predictions))
print("Vorhergesagte Klasse:", prediction)

In [ ]:
plt.plot()
plt.imshow(np.reshape(X_test.iloc[data_index].values, (8,8)), cmap=plt.cm.gray)
plt.title(f"Ziffer: {y_test.iloc[data_index]}", fontsize = 20)

Wie wir sehen stimmt die vorhergesagte Ziffer in diesem Fall mit der tatsächlichen Ziffer überein. Im nächsten Schritt schauen wir uns die Vorhersage über alle Testdatenpunkte an und schauen auf die Accuarcy.
Dazu sagen wir mit jedem der 10 Modelle auf jedem Testdatenpunkt vorher, und merken uns die Klasse mit der höchsten Wahrscheinlichkeit:

In [ ]:
test_predictions = []
for data_index in range(0, len(X_test)):
    data_point = X_test.iloc[[data_index]]
    predictions = []
    for model in models:  
        prediction = model.predict_proba(data_point)[0][1]
        predictions.append(prediction)
    predicted_class = predictions.index(max(predictions))
    test_predictions.append(predicted_class)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, test_predictions)

Wie wir sehen können wir mit unserem recht simplen Modell bereits 95% der Fälle richtig vorhersagen.

Zum Schluss schauen wir uns die Confusion Matrix der einzelnen Fälle an und den Classificatin Report:

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
cm = confusion_matrix(y_test, test_predictions, labels=range(0,10))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=range(0,10))
disp.plot()

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, test_predictions))

Zum Schluss schauen wir uns den Fall an, wenn wir direkt auf allen Labels trainieren:

In [ ]:
logReg = LogisticRegression(max_iter=10000)
logReg.fit(X_train, y_train);

In [ ]:
accuracy_score(y_test, logReg.predict(X_test))

Wir wir sehen kann die Klasse `LogisticRegression` auch direkt mit mehreren Labels umgehen.